We will use [Curve P-384](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.186-4.pdf)  as an example:

$p = 2^{384} - 2^{128} - 2^{96} + 2^{32} - 1$

Points satisfy the following equation:

$y^2 = x^3 + Ax + B \mod p$

with $A$ and $B$ as below. 

In [1]:
p = 2**384 - 2**128 - 2**96 + 2**32 - 1
Fp = FiniteField(p)
A = -3
B = 0xb3312fa7e23ee7e4988e056be3f82d19181d9c6efe8141120314088f5013875ac656398d8a2ed19d2a85c8edd3ec2aef
E = EllipticCurve(Fp,[A,B])
print(E)

Elliptic Curve defined by y^2 = x^3 + 39402006196394479212279040100143613805079739270465446667948293404245721771496870329047266088258938001861606973112316*x + 27580193559959705877849011840389048093056905856361568521428707301988689241309860865136260764883745107765439761230575 over Finite Field of size 39402006196394479212279040100143613805079739270465446667948293404245721771496870329047266088258938001861606973112319


In [3]:
def GenPubKey(A, B, p, xP, yP):
    
    Fp = FiniteField(p)
    E = EllipticCurve(Fp,[A,B])
    
    P = (xP, yP)
    nA = ZZ(Fp.random_element()) # how to chose this secret multiplier? Sample from eliptic curve?
    
    Qa = tuple([nA*x for x in P]) # nA * P
    
    return Qa, nA

def Encript(A, B, p, xP, yP, Qa, m1, m2):
    
    Fp = FiniteField(p)
    E = EllipticCurve(Fp,[A,B])
    
    P = (xP, yP)
    k = ZZ(Fp.random_element())
    S =  tuple([k*x for x in Qa]) # k * Qa
    xS = S[0]
    yS = S[1]

    
    R = tuple([k*x for x in P]) # k * P
    c1 = (xS*m1) % p
    c2 = (yS*m2) % p
    
    return R, c1, c2

def Decript(A, B, p, xP, yP, R, c1, c2):
    P = (xP, yP)
    T = tuple([nA*x for x in R]) # nA * R
    
    xT = T[0]
    yT = T[1]
    
    m1 = ((xT ^ -1) * c1) % p
    m2 = ((yT ^ -1) * c2) % p
    
    return m1, m2

In [4]:
xP, yP = 3, 4
m1, m2 = 5, 6

Qa, nA = GenPubKey(A, B, p, xP, yP)

R, c1, c2 = Encript(A, B, p, xP, yP, Qa, m1, m2)

rm1, rm2 = Decript(A, B, p, xP, yP, R, c1, c2)

In [5]:
# Prove that methods work

assert((m1, m2) == (rm1, rm2))